In [8]:
import pandas as pd
import numpy as np

file_path = "D:/Python/3SP25/CHOTOT_motorcycles.csv"
df = pd.read_csv(file_path)

df['name'] = df['Brand'] + ' ' + df['Model']
df["name"], name_labels = pd.factorize(df["name"])
df['Price'] = df['Price'].apply(lambda x: int(x.replace('.', '').replace(' đ', '').replace('đ', '').strip()))
df['Showroom_Price'] = pd.to_numeric(df['Min_Price'].str.lower().str.replace('triệu', '').str.strip(), errors='coerce') * 1000000
df['Kilometer Driven'] = df['Kilometer Driven'].replace('Không có thông tin', np.nan)
df['Kilometer Driven'] = pd.to_numeric(df['Kilometer Driven'], errors='coerce')
df['Year of Manufacture'] = df['Year of Manufacture'].str.extract(r'(\d{4})')
df['Year of Manufacture'] = pd.to_numeric(df['Year of Manufacture'], errors='coerce')
for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = df[col].fillna(df[col].mean())

df = df.drop(['Min_Price', 'Brand', 'Model', 'Địa điểm', 'Thời gian đăng'], axis=1)

# print(df.head())

output_path = "BIKE_DETAILS.csv"
df.to_csv(output_path, index=False)
print("Done.")

Done.


In [10]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor

import joblib

df = pd.read_csv('D:/Python/3SP25/BIKE_DETAILS.csv')

X = df.drop(columns=['Price'])
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'R2 Score: {r2}')

joblib.dump(model, 'bike_price_model.pkl')
joblib.dump(name_labels, 'name_labels.pkl')

MSE: 688247356738865.4
R2 Score: 0.43956372730660775


['name_labels.pkl']

In [13]:
model = joblib.load('bike_price_model.pkl')
name_labels = joblib.load('name_labels.pkl')

def preprocess_input(user_input):
    df_input = pd.DataFrame([user_input])

    df_input['name'] = name_labels.get_indexer([df_input['name'][0]])[0]

    df_input['Year of Manufacture'] = pd.to_numeric(df_input['Year of Manufacture'], errors='coerce')
    df_input['Kilometer Driven'] = pd.to_numeric(df_input['Kilometer Driven'], errors='coerce')

    df_input = df_input.fillna(0)

    return df_input

def predict_price(data_input):
    processed_input = preprocess_input(data_input)
    prediction = model.predict(processed_input)
    return prediction[0]

data_input = {
    'Year of Manufacture': '2022',
    'Kilometer Driven': '5110',
    'name': 'Honda SH',
    'Showroom_Price': '78000000',
}

price = predict_price(data_input)
print(f"Giá xe dự đoán: {price:,.0f} VNĐ")

Giá xe dự đoán: 72,952,000 VNĐ
